In [1]:
from pymongo import MongoClient
from collections import deque, defaultdict

# Connect to MongoDB
client = MongoClient(os.getenv("MONGO_URI"))
db = client["virtualgenesis"]  

collection_name = "jarvis_swap"
token_name = "JARVIS"
collection = db[collection_name]

# Fetch and sort transactions
cursor = collection.find().sort("timestamp", 1)
trades_by_maker = defaultdict(list)

# Prepare trade data grouped by trader
for doc in cursor:
    maker = doc.get("maker")
    swap_type = doc.get("swapType")
    try:
        price = float(doc.get("genesis_usdc_price", 0))
    except (TypeError, ValueError):
        continue

    if not price or not maker:
        continue

    if swap_type == "buy":
        try:
            # What user pays for (before tax) - full purchase amount
            amount_bought_before_tax = float(doc.get("JARVIS_OUT_BeforeTax", 0))
            # What user actually receives in wallet (after tax)
            amount_received = float(doc.get("JARVIS_OUT_AfterTax", 0))
        except (TypeError, ValueError):
            continue
        
        if amount_bought_before_tax <= 0:
            continue
            
        trades_by_maker[maker].append({
            'type': 'buy',
            'amount_bought_before_tax': amount_bought_before_tax,  # What user paid for
            'amount_received': amount_received,  # Actual tokens in wallet
            'price': price
        })

    elif swap_type == "sell":
        try:
            # What user actually gets proceeds from (after tax deduction)
            amount_sold_net = float(doc.get("JARVIS_IN_AfterTax", 0))
            # What actually left user's wallet (before tax or after tax + tax)
            amount_from_wallet = float(doc.get("JARVIS_IN_BeforeTax", 0))
        except (TypeError, ValueError):
            continue
            
        if amount_sold_net <= 0:
            continue
            
        trades_by_maker[maker].append({
            'type': 'sell',
            'amount_sold_net': amount_sold_net,  # Net proceeds after tax
            'amount_from_wallet': amount_from_wallet,  # Tokens that left wallet
            'price': price
        })

# Store final results
results = []

# Calculate PnL per trader
for maker, trades in trades_by_maker.items():
    buy_queue = deque()  # Stores tokens actually received (after tax)
    total_realized_pnl = 0.0

    for trade in trades:
        if trade['type'] == 'buy':
            # Store actual tokens received (what's in wallet)
            buy_queue.append({
                'amount': trade['amount_received'],  # Tokens actually in wallet
                'amount_paid_for': trade['amount_bought_before_tax'],  # What user paid for
                'price': trade['price']
            })

        elif trade['type'] == 'sell':
            amount_from_wallet = trade['amount_from_wallet']  # Tokens leaving wallet
            amount_sold_net = trade['amount_sold_net']  # Net proceeds after tax
            sell_price = trade['price']

            # Match against oldest buys (FIFO) - match what's leaving wallet
            remaining_to_match = amount_from_wallet
            
            while remaining_to_match > 0 and buy_queue:
                buy = buy_queue.popleft()
                buy_amount_in_wallet = buy['amount']  # Tokens actually owned
                buy_amount_paid_for = buy['amount_paid_for']  # What user originally paid for
                buy_price = buy['price']

                # How much can we match?
                matched_amount_in_wallet = min(remaining_to_match, buy_amount_in_wallet)
                
                # Calculate the corresponding amount user originally paid for
                ratio = matched_amount_in_wallet / buy_amount_in_wallet
                matched_amount_paid_for = buy_amount_paid_for * ratio

                # PnL = What user paid for - What user gets from selling
                # User paid: matched_amount_paid_for × buy_price
                # User gets: (matched_amount_in_wallet / amount_from_wallet) × amount_sold_net × sell_price
                proceeds_ratio = matched_amount_in_wallet / amount_from_wallet
                actual_proceeds = amount_sold_net * sell_price * proceeds_ratio
                cost_paid = matched_amount_paid_for * buy_price
                
                pnl = actual_proceeds - cost_paid
                total_realized_pnl += pnl

                # Update remaining amounts
                remaining_to_match -= matched_amount_in_wallet
                remaining_buy_amount = buy_amount_in_wallet - matched_amount_in_wallet

                # If buy order partially filled, put remainder back
                if remaining_buy_amount > 0:
                    remaining_paid_for = buy_amount_paid_for * (remaining_buy_amount / buy_amount_in_wallet)
                    buy_queue.appendleft({
                        'amount': remaining_buy_amount,
                        'amount_paid_for': remaining_paid_for,
                        'price': buy_price
                    })

    # Calculate remaining inventory (tokens still held)
    remaining_tokens = sum(b['amount'] for b in buy_queue)

    results.append({
        'trader': maker,
        'token': token_name,
        'net_pnl': round(total_realized_pnl, 4),
        'remaining_tokens': round(remaining_tokens, 4)
    })

# Print results
for row in results:
    print("="*60)
    print(f"Trader: {row['trader']}")
    print(f"Token: {row['token']}")
    print(f"Net PnL: ${row['net_pnl']}")
    print(f"Remaining Tokens: {row['remaining_tokens']}")
    print("="*60)

Trader: 0x6acA0AEFA3CF63a67282a26B8E3Db08e03c9483C
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 15292674.6132
Trader: 0x4240398c6a07147B062376e4a0A8562F8340c12D
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 5678699.7276
Trader: 0xE798045BECBBdCe8031d28e8E61Cb3c56e6Ee8bF
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 4879451.0125
Trader: 0x6Fec65619e49378220D9353D8F05f8eA74D93cb6
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 1785663.6475
Trader: 0x0289c426c6804d648427cE2D67C35d4C309a4eC4
Token: JARVIS
Net PnL: $-65.4841
Remaining Tokens: 213.1041
Trader: 0x270028e8CfE2c9D5A8F94c09ced11314999aD019
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 9212.2474
Trader: 0x83fffF6f6bd73c63E162a348b24A762a1cA0361f
Token: JARVIS
Net PnL: $-113.2562
Remaining Tokens: 0
Trader: 0xFA3a8AAf8583872f9961954D5e6Eb88fc697EFC2
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 4608359.5654
Trader: 0x2B3b2F3C3B1Fa15e26fd5b9ceee2958d07Ffd260
Token: JARVIS
Net PnL: $0.0
Remaining Tokens: 186.4138
Trader: 0xf1A14825Ab